In [2]:
# import pandas, shutil, os
import pandas as pd
import shutil
import os
import random
import math
import glob

In [3]:
# allow multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print("Multiple outputs per cell")

Multiple outputs per cell


In [12]:
# variables to determine the source and destination paths
TRAIN_DIRECTORY = 'C:/Users/akash/Desktop/Akash/MSOEML/NIH_data/images' # no need to put a forward slash at the front of directory name

TEST_DIRECTORY = 'C:/Users/akash/Desktop/Akash/MSOEML/NIH_data/imagesTest'

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1540382389.py, line 2)

In [5]:
# read in data and inspect it
images = pd.read_csv('NIH.csv')

In [6]:
# make the Image Index column as the index
images.set_index('Image Index', inplace=True)
images.head()

,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
Image Index,,,,,,,,,,
00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168


In [7]:
# filter out multiple conditions & print the remaining ones
images.drop(images[images['Finding Labels'].str.contains('\|')].index, inplace=True)

diseases = images['Finding Labels'].unique()
diseases

array(['Cardiomegaly', 'No Finding', 'Hernia', 'Infiltration', 'Nodule',
       'Emphysema', 'Effusion', 'Atelectasis', 'Mass', 'Pneumothorax',
       'Pleural_Thickening', 'Fibrosis', 'Consolidation', 'Edema',
       'Pneumonia'], dtype=object)

In [11]:
for disease in diseases:
    

for disease in diseases:
    if not os.path.exists(TEST_DIRECTORY + '/' + disease):
        os.makedirs(TEST_DIRECTORY + '/' + disease)

print("Confirming that the directories exist")

print("Training Directory")
print(glob.glob(f'{TRAIN_DIRECTORY}/*/'))
print("Validation Directory")
print(glob.glob(f'{TEST_DIRECTORY}/*/'))

Confirming that the directories exist
Training Directory
['/Volumes/90OL67YGN/images\\Atelectasis\\', '/Volumes/90OL67YGN/images\\Cardiomegaly\\', '/Volumes/90OL67YGN/images\\Consolidation\\', '/Volumes/90OL67YGN/images\\Edema\\', '/Volumes/90OL67YGN/images\\Effusion\\', '/Volumes/90OL67YGN/images\\Emphysema\\', '/Volumes/90OL67YGN/images\\Fibrosis\\', '/Volumes/90OL67YGN/images\\Hernia\\', '/Volumes/90OL67YGN/images\\Infiltration\\', '/Volumes/90OL67YGN/images\\Mass\\', '/Volumes/90OL67YGN/images\\No Finding\\', '/Volumes/90OL67YGN/images\\Nodule\\', '/Volumes/90OL67YGN/images\\Pleural_Thickening\\', '/Volumes/90OL67YGN/images\\Pneumonia\\', '/Volumes/90OL67YGN/images\\Pneumothorax\\']
Validation Directory
['/Volumes/90OL67YGN/imagesTest\\Atelectasis\\', '/Volumes/90OL67YGN/imagesTest\\Cardiomegaly\\', '/Volumes/90OL67YGN/imagesTest\\Consolidation\\', '/Volumes/90OL67YGN/imagesTest\\Edema\\', '/Volumes/90OL67YGN/imagesTest\\Effusion\\', '/Volumes/90OL67YGN/imagesTest\\Emphysema\\', '/

In [9]:
for fileName in os.listdir(TRAIN_DIRECTORY):
    if fileName in diseases:
        continue
    if fileName not in images.index:
        continue
    
    # move the file to the appropriate directory
    sp = os.path.join(TRAIN_DIRECTORY, fileName)
    disease = images.loc[fileName]['Finding Labels']
    dp = os.path.join(TRAIN_DIRECTORY, disease)
    _ = shutil.move(sp, dp)